# 🏥 Medical Image Analysis with Google MedGemma 1.5

[![Python](https://img.shields.io/badge/Python-3.10+-blue.svg)](https://www.python.org/downloads/) 
[![PyTorch](https://img.shields.io/badge/PyTorch-2.6+-red.svg)](https://pytorch.org/) 
[![Transformers](https://img.shields.io/badge/🤗-Transformers-yellow.svg)](https://huggingface.co/transformers/)

---

## 🎯 Overview

This notebook demonstrates **Google's MedGemma 1.5** model for analyzing medical images including X-rays, CT scans, and ultrasounds.

### Key Capabilities:
- 🔍 Analyze chest X-rays and identify abnormalities
- 🩺 Detect pulmonary conditions (pneumonia, edema, effusions)
- 📊 Process ultrasound imaging
- 💡 Provide clinical context-aware assessments

### ⚠️ Important Disclaimer
> **Educational purposes only.** AI outputs should NOT replace professional medical diagnosis. Always consult qualified healthcare professionals.

---

## 📦 Setup & Installation

Installing required packages: PyTorch, Transformers, Pillow, and dependencies.

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers accelerate
!pip install pillow requests
!pip install numpy

## ✅ Environment Verification

Checking PyTorch version, CUDA availability, and GPU device.

In [ ]:
import torch
print(torch.__version__)          # >= 2.6.0
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # NVIDIA

2.9.1+cu128
True
NVIDIA H100 80GB HBM3


## 🔐 Hugging Face Authentication

Login to Hugging Face to access the MedGemma model. Get your token from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

In [ ]:
from huggingface_hub import login
login(token="")  # Replace with your Hugging Face token

/root/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 🚀 Load MedGemma Model

Loading the **google/medgemma-1.5-4b-it** model with BFloat16 precision for GPU acceleration.

In [4]:
from transformers import pipeline
from PIL import Image
import requests
import torch

pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-1.5-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda",
)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 15.91it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


---

# 🔬 Medical Imaging Analysis

---

## 📸 Case 1: Chest X-Ray Analysis

**Objective:** Analyze a standard chest X-ray and identify any abnormalities.

**Key structures to evaluate:** Lung fields, heart size, mediastinum, bony structures

In [6]:
# Image attribution: Stillwaterising, CC0, via Wikimedia Commons
image_url = "https://www.cvmg.com/wp-content/uploads/2014/10/chest-xray.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Describe this X-ray, whats wrong here?"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the provided chest X-ray, here's a description and assessment:

**Description:**

*   The image is a standard AP (anteroposterior) chest X-ray.
*   The lungs appear clear with normal expansion.
*   The heart size seems within normal limits.
*   The mediastinum (the space between the lungs containing the heart, great vessels, trachea, etc.) appears normal in width.
*   The bony structures of the rib cage are intact.
*   The diaphragm is visible and appears normal in position.

**Potential Issues/Observations:**

*   There is a small, nodular opacity in the right upper lobe of the lung. This could be a benign lesion (such as a granuloma or hamartoma) or an early sign of a more serious condition. Further investigation may be warranted depending on the clinical context.

**Disclaimer:**

I am an AI and cannot provide medical diagnoses. This is not a substitute for a professional medical evaluation by a qualified radiologist or physician. A proper diagnosis requires a complete clin

## 🫁 Case 2: Suspected Pneumonia

**Patient:** 58-year-old male

**Clinical History:**
- 3-month progressive shortness of breath
- Non-productive cough
- 7 kg weight loss over 8 weeks
- Low-grade fevers and night sweats
- No response to multiple antibiotic courses

**Differential:** Pneumonia, tuberculosis, lung malignancy, interstitial lung disease

In [8]:
image_url = "https://radiologyassistant.nl/assets/chest-x-ray-lung-disease/a50e9a848bc956_5a-NHL.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "A 58-year-old male with a 3-month history of progressive shortness of breath and non-productive cough presents for evaluation. He reports persistent fatigue, a 7-kg (15-lb) unintentional weight loss over the past 8 weeks, and intermittent low-grade fevers. He denies significant chest pain but notes occasional night sweats and decreased exercise tolerance. There has been no significant response to multiple courses of antibiotics prescribed for presumed lower respiratory tract infection.Describe this X-ray, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the chest X-ray, the following observations are made:

*   **Right Lung:** There appears to be consolidation, possibly opacification or haziness, in the right upper and lower lung fields. The consolidations are patchy.
*   **Left Lung:** The left lung appears relatively clear.
*   **Heart Size:** The heart size seems normal.
*   **Mediastinum:** The mediastinum is unremarkable.
*   **Bones:** The bony structures appear intact.

**Conclusion:**

The X-ray findings are concerning for **pneumonia**. The patchy opacities in the right lung suggest inflammation and fluid accumulation within the lung tissue. While the history of cough, shortness of breath, and fatigue is suggestive of pulmonary infection, the progressive nature of the symptoms, weight loss, night sweats, and lack of response to antibiotics raise concerns for a more complex underlying cause. The patient should be further investigated to determine the precise diagnosis, which could be a complicated pneumonia, lung absc

## 💨 Case 3: Acute Respiratory Distress

**Patient:** 62-year-old

**Clinical History:**
- Acute onset dyspnea (past few days)
- Pleuritic chest pain
- Recent prolonged immobility (long trip)
- Intermittent lightheadedness

**Risk Factors:** DVT/PE risk from immobility

**Differential:** Pulmonary embolism, pleural effusion, pneumothorax

In [12]:
image_url = "https://radiologyassistant.nl/assets/chest-x-ray-lung-disease/a50e9d58ded69a_8-Longinfarct.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text","text": "A 62-year-old patient presents with acute onset of worsening shortness of breath over the past few days. The dyspnea began suddenly and has progressively limited physical activity. The patient also reports pleuritic chest discomfort, especially on deep inspiration, and a dry cough. There is a recent history of prolonged immobility after a long trip and intermittent episodes of lightheadedness. No fever, chills, or significant productive cough are noted.Describe this X-ray, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the provided chest X-ray, here is a concise description and assessment:

**X-ray Description:**

The image shows a frontal chest X-ray with a side view on the right.

**Findings:**

*   **Lungs:** The lungs appear relatively clear bilaterally.
*   **Pleura:** There is a significant pleural effusion on the right side, obscuring the right hemidiaphragm.
*   **Heart/Mediastinum:** The heart size appears within normal limits. The mediastinal structures are unremarkable.
*   **Bones:** The bony structures of the rib cage are visible.

**Possible Explanation**

Based on the presence of a large right-sided pleural effusion, a potential cause of shortness of breath and pleuritic chest discomfort is present. A possible underlying cause of this is a **pneumonia**, **tumor**, or **heart failure**.

**Further Evaluation**

To confirm the diagnosis and determine the cause, additional imaging or clinical data may be needed. Consider performing a CT scan of the chest to better evaluate the e

## ❤️ Case 4: Suspected Heart Failure

**Patient:** 68-year-old

**Clinical History:**
- Progressive dyspnea (several weeks)
- Orthopnea (SOB when lying flat)
- Nocturnal cough with breathlessness
- Lower limb swelling

**Classic HF Features:** ✅ Orthopnea ✅ PND ✅ Peripheral edema

**Expected Findings:** Cardiomegaly, pulmonary edema, bilateral infiltrates

In [14]:
image_url = "https://radiologyassistant.nl/assets/chest-x-ray-lung-disease/a509797a678f8e_CHF-1c1.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text","text": "A 68-year-old individual presents to the clinic with progressively worsening shortness of breath over the past several weeks. The dyspnea is present at rest and worsens with exertion. The patient also reports intermittent orthopnea (shortness of breath when lying flat) and nocturnal cough with brief episodes of waking up breathless at night. There is associated fatigue and decreased exercise tolerance. The patient denies chest pain, hemoptysis, or high fever. There has been a gradual increase in lower limb swelling over the last few weeks.Describe this X-ray, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


This chest X-ray shows bilateral pulmonary infiltrates, with a predominantly perihilar distribution. The heart size appears enlarged.

The overall presentation is suggestive of pulmonary edema, a condition where there is an excess fluid buildup in the lungs. Possible causes of pulmonary edema include heart failure, fluid overload, kidney disease, and pneumonia.

Given the patient's symptoms of shortness of breath, orthopnea, nocturnal cough, and lower limb swelling, heart failure is a strong consideration.

It would be important to consider other potential causes, such as pneumonia or fluid overload, and to correlate these findings with the patient's clinical history and physical exam. Further evaluation, including cardiac function testing, blood work, and possibly a pulmonary function test, would be required to determine the underlying etiology.
Final Answer: The final answer is $\boxed{Pulmonary edema}$


## 🫁 Case 5: Chronic Progressive Respiratory Symptoms

**Patient:** 60-year-old

**Clinical History:**
- Progressive dyspnea and dry cough (weeks)
- Unintentional weight loss (2-3 months)
- No relief with antibiotics ⚠️

**Red Flags:** Weight loss → malignancy, No antibiotic response → non-infectious

**Differential:** Interstitial lung disease, bronchioloalveolar carcinoma, chronic infection

In [15]:
image_url = "https://radiologyassistant.nl/assets/chest-x-ray-lung-disease/a50e999f9d1cae_3-diffuse-BAC.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text","text": "A 60-year-old patient presents with several weeks of progressively worsening shortness of breath and a persistent dry cough. The symptoms began insidiously and have gradually limited the patient’s ability to perform daily activities. The patient also reports occasional mild chest discomfort, fatigue, and a small amount of unintentional weight loss over the past 2–3 months. There is no significant fever or chills. No significant relief with courses of previous antibiotics.Describe this X-ray, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the X-ray provided, here is a description and potential diagnosis:

**X-ray Description:**

The image shows a chest X-ray. The left lung field appears opaque or dense, with some haziness visible. The right lung field also appears somewhat hazy, although less dense than the left. There's a possible opacity in the left lower lobe.

**Potential Problem:**

The findings on the X-ray suggest a possible **pneumonia** in the left lung, potentially with some associated edema. The left lung appears denser and has some haziness, indicating consolidation or fluid.

**Important Considerations:**

*This information is not a substitute for a medical diagnosis from a healthcare professional. A proper diagnosis requires a comprehensive evaluation, including the patient's medical history, physical examination, and possibly additional tests.
* Further investigations, such as a CT scan of the chest, would be needed to determine the cause of the opacity and assess the extent of the disease.

**Di

---

# 🔊 Ultrasound Imaging Analysis

---

## 🩺 Case 6: Uterine Fibroid

**Patient:** 38-year-old female

**Clinical History:**
- Worsening menstrual symptoms (1 year)
- Heavy bleeding with clots
- Pelvic pressure and discomfort
- Fatigue

**Suspected:** Uterine fibroids (leiomyomas)

In [20]:
# image_url = ""
image = Image.open("/root/Subserosal-uterine-fibroid.jpg")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "A 38-year-old woman presents with a history of progressively worsening menstrual symptoms over the past year. She reports heavy and prolonged menstrual bleeding, sometimes associated with passage of clots. The patient also complains of lower abdominal fullness, intermittent pelvic discomfort, and a sensation of pressure that is more noticeable during menstruation. Over the past few months, she has noticed increasing fatigue and reduced exercise tolerance. She denies acute pelvic pain, fever, or abnormal vaginal discharge. There is no history of sudden onset symptoms.Describe this altrasound, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


This ultrasound shows a thickened uterus and a possible fibroid within it.

Based on the image:

*   **Thickened Uterus:** The uterus appears more prominent than normal, suggesting it may be enlarged.
*   **Possible Fibroid:** A mass/cyst may be present within the uterus (as indicated by the "L" and "2" markers). Fibroids are common benign growths in the uterus.

The symptoms described are consistent with the possible presence of a fibroid.

Given the patient's symptoms, further evaluation may be necessary to confirm the diagnosis and determine the best course of treatment. This may include additional imaging or consultation with a gynecologist.

Note: This is a preliminary interpretation based solely on the provided image and clinical context. A definitive diagnosis requires a complete examination and potentially further imaging or procedures. The information provided should not be used as a substitute for professional medical advice.


## 🤰 Case 7: Routine Antenatal Ultrasound

**Patient:** 27-year-old pregnant female

**Visit:** Routine mid-pregnancy evaluation

**Status:** Uncomplicated pregnancy, regular fetal movements, no concerns

In [21]:
image = Image.open("/root/ultrasound-of-baby-1080x780-1.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "A 27-year-old pregnant woman presents for a routine antenatal evaluation during the mid-pregnancy period. She reports feeling generally well with no acute complaints. Fetal movements have been perceived regularly over the past few weeks. There is no history of vaginal bleeding, fluid leakage, abdominal trauma, or fever. The pregnancy has been uncomplicated so far.Describe this altrasound, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


This ultrasound shows a fetus in the second trimester. The fetus appears generally normal in appearance, but there is a concerning feature:  the placenta seems to be positioned abnormally, possibly toward the fundus (top) of the uterus. This could be contributing to the fetus appearing to be closer to the top of the image. It is possible that an ultrasound technician, or doctor may need to repeat the ultrasound to confirm this.

A routine evaluation like this would normally be performed to assess the baby's growth, development, and well-being, as well as the mother's health. If the placenta is positioned abnormally, the doctor may recommend closer monitoring or further investigation to assess potential complications, like issues with blood flow or growth. Further investigation would be needed for a definitive diagnosis.



## 🔍 Case 8: Flank Pain Evaluation

**Patient:** 45-year-old

**Clinical History:**
- Intermittent right-sided flank pain (months)
- Urinary urgency and dysuria
- No fever or hematuria

In [22]:
image = Image.open("/root/ultrasound-of-baby-1080x780-1.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "A 45-year-old patient presents with a several-month history of intermittent flank pain localized to the right side. The pain is described as dull and aching, sometimes increasing in intensity after physical activity. The patient also reports frequent episodes of urinary urgency and discomfort during urination. There is no history of fever, chills, or gross hematuria.Describe this altrasound, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the ultrasound image, here's a description and potential explanation for the patient's symptoms:

**Ultrasound Description:**

The image shows a fetus in the second or third trimester. The fetus appears to be in a relatively normal position, and the major structures (heart, brain, limbs) are visible. The image is clear and shows the typical appearance of a fetus at this stage.

**Potential Explanation for Symptoms:**

The patient's symptoms (flank pain, urinary urgency, and discomfort during urination) could be related to a **urinary tract infection (UTI)**, potentially involving the **kidney** (pyelonephritis). UTIs are more common in older adults and can present with flank pain, urinary symptoms, and sometimes fever or chills.

**Important Considerations:**

*   **Diagnosis:** This is an ultrasound image. It is important to note that a diagnosis of UTI cannot be made from this image alone. Further investigation, such as a urinalysis and urine culture, is necessary to confirm

In [23]:
image = Image.open("/root/AdobeStock_259889262.webp")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "A 45-year-old patient presents with a several-month history of intermittent right-sided flank pain. The pain is described as dull and aching in nature, with occasional episodes of increased intensity, particularly after physical activity or prolonged standing. The pain does not radiate and is partially relieved with rest and over-the-counter analgesics.The patient also reports urinary symptoms, including increased urinary urgency and dysuria over the same period. There is no associated gross hematuria, fever, chills, or night sweats. The patient denies nausea, vomiting, or bowel habit changes.There is no prior history of renal stones, recurrent urinary tract infections, or known urological disorders. The patient has not noticed any recent weight loss or loss of appetite.Describe this altrasound, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the provided image, it is impossible to determine what is wrong. The image appears to be a series of CT scans of the head, not an ultrasound. CT scans are used to visualize structures within the body, such as the brain, bones, and organs. An ultrasound would be a different type of imaging, which is a sound wave based technique.
If you have a specific medical concern or are trying to diagnose a condition, you should consult a healthcare professional for accurate diagnosis and treatment.
I am an AI chatbot and not a medical professional.


---

# 📊 Conclusion

## ✅ Key Takeaways

**MedGemma 1.5 Capabilities:**
- ✓ Multimodal understanding (image + text)
- ✓ Clinical context awareness
- ✓ Multiple imaging modalities
- ✓ Detailed anatomical descriptions

## ⚠️ Limitations
- **Not a diagnostic tool** - requires professional validation
- **Educational purpose only**
- **Clinical judgment essential**

## 📚 References
- [MedGemma Model](https://huggingface.co/google/medgemma-1.5-4b-it)
- [Radiology Assistant](https://radiologyassistant.nl/)

---

**Remember:** AI augments, not replaces, clinical expertise. Always consult healthcare professionals.